# Research: Exploring free responses
This notebook demonstrates ways of modifying instructions for free response questions in EDSL. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/free_responses.ipynb)

In [1]:
# ! pip install edsl

## Creating free response questions

In [2]:
from edsl.questions import QuestionFreeText

q1 = QuestionFreeText(
    question_name = "color",
    question_text = "What is your favorite color? {{ instruction }}"
)

q2 = QuestionFreeText(
    question_name = "number",
    question_text = "What is your favorite number? {{ instruction }}"
)

## Parameterizing the questions with special instructions

In [3]:
from edsl import Scenario

instructions = [
    "Be as specific as possible.",
    "Be concise.",
    "Be as short as possible."
]

scenarios = [Scenario({"instruction":i}) for i in instructions]

## Creating agent personas with specific survey contexts

In [4]:
from edsl import Agent

ages = [
    "You are middle-aged.",
    "You are a senior citizen.",
    "You are a young adult."
]

contexts = [
    "You are handwriting answers in a paper survey.",
    "You are typing answers in an online survey.",
    "You are providing answers verbally to a researcher in a live interview."
]

agents = [Agent(traits={"age":a, "context":c}) for a in ages for c in contexts]

## Selecting LLMs

In [5]:
from edsl import Model

models = [Model(m) for m in ['gpt-3.5-turbo', 'gpt-4-1106-preview']]

## Administering the survey

In [6]:
from edsl import Survey 

survey = Survey([q1, q2])

results = survey.by(scenarios).by(agents).by(models).run()

In [7]:
results.select("model.model", "age", "context", "scenario.*", "color", "number").print()

model.model,agent.age,agent.context,scenario.instruction,answer.color,answer.number
gpt-3.5-turbo,You are middle-aged.,You are handwriting answers in a paper survey.,Be concise.,Blue,My favorite number is 7.
gpt-3.5-turbo,You are middle-aged.,You are typing answers in an online survey.,Be as short as possible.,Blue,7
gpt-3.5-turbo,You are middle-aged.,You are providing answers verbally to a researcher in a live interview.,Be concise.,My favorite color is blue.,My favorite number is 7.
gpt-3.5-turbo,You are middle-aged.,You are typing answers in an online survey.,Be as specific as possible.,My favorite color is deep navy blue.,My favorite number is 7 because it's considered lucky in many cultures.
gpt-3.5-turbo,You are a young adult.,You are providing answers verbally to a researcher in a live interview.,Be concise.,My favorite color is blue.,My favorite number is 7.
gpt-3.5-turbo,You are a young adult.,You are handwriting answers in a paper survey.,Be as short as possible.,Blue,7
gpt-3.5-turbo,You are middle-aged.,You are typing answers in an online survey.,Be concise.,Blue,My favorite number is 7.
gpt-3.5-turbo,You are a senior citizen.,You are handwriting answers in a paper survey.,Be as specific as possible.,My favorite color is sky blue.,My favorite number is 7 because it's considered lucky in many cultures.
gpt-3.5-turbo,You are a senior citizen.,You are typing answers in an online survey.,Be concise.,My favorite color is blue.,My favorite number is 7.
gpt-3.5-turbo,You are middle-aged.,You are handwriting answers in a paper survey.,Be as short as possible.,Blue,My favorite number is 7.


---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>